# Init

In [1]:
from random import random

import gym
from gym import wrappers

import numpy as np
from scipy.spatial import KDTree

# Env Settings

In [2]:
_OBS_DIM = 4
_ACT_DIM = 1
_BIGNUM  = 1e5

# Minimum Viable Components

In [3]:
K     = 3 # ---------------------- Number of neighbors to query
N     = 0 # ---------------------- Number of exemplars
F     = np.zeros( (N,_OBS_DIM,) ) # Approximating function
V     = np.array( [] ) # ------- Action values
A     = np.zeros( (N,_ACT_DIM,) ) # Actions (Output)
KDT   = None # ------------------- Spatial tree
eps   = 1.0 # -------------------- Exploration probability
decay = 0.99
rad   = 0.125 # ------------------ Minimum distance between exemplars (Overwrite radius)
vMar  = 0.10 # ------------------- Allowed margin on value estimates

In [4]:
def points_from_indices( pnts, ndcs ):
    """ Get the subset of `pnts` designated by `ndcs` """
    N = len( ndcs )
    P = np.zeros( (N,pnts.shape[1],) )
    for i, idx in enumerate( ndcs ):
        P[i,:] = pnts[idx,:]
    return P

In [5]:

def cartpole_reward( X ):
    """ Reward high speed at the bottom and low speed at the top """
    # 0. Set limits
    maxThetaDot =  10.0
    maxX        =   2.0
    # 1. Set weights
    thFactor    = 100.0
    thDotFactor =   8.0
    # 2. Unpack & Normalize state
    xN        = abs( X[0] ) # Fulcrum position
    thetaN    = X[2] # ------ Angle
    thetaDotN = abs( X[3] ) # Angular velocity
    # 3. Reward high speed at the bottom and low speed at the top
    R = thFactor*np.cos(thetaN) - thDotFactor*np.cos(thetaN)*(thetaDotN)
    # 4. Adjust for wandering
    if xN > maxX:
        R -= xN
    return R
    

def add_particle( state, action, value, getTree = False ):
    """ Add a new particle to the value function """
    global N, F, V, A, KDT
    
    # print( f"V-Stack: {F.shape} + {state.shape}" )
    
    if N < 1:
        F = state
        V = np.array( [value,] )
        A = action
    else:
        F = np.vstack( (F,state, ) )
        temp = V.tolist()
        temp.append( value )
        V = np.array( temp )
        A = np.vstack( (A,action,) )

    # print( f"New Values: {V}" )
    
    N += 1
    if getTree:
        return N, KDTree( F )
    else:
        return N, None
        

def recalc_spatial_tree():
    """ Recalculate spatial tree """
    global N, F, V, A, KDT
    if N > 1:
        KDT = KDTree( F )

    
def get_action_and_value_inv_dist( state ):
    """ Estimate the current optimal action and value for the state """
    global N, F, V, A, KDT
    if (N < 1) or (KDT is None):
        return None, None
    dists, indcs = KDT.query( state, K )
    dists = dists[0]
    indcs = indcs[0]
    # print( dists, indcs )
    fractV = []
    indcsV = []
    for i, d in enumerate( dists ):
        # print( d )
        if d < _BIGNUM:
            fractV.append( 1.0/d )
            indcsV.append( indcs[i] )
    fractV = np.array( fractV )
    normD  = np.linalg.norm( fractV )
    fractV = fractV / normD
    rtnAct = np.zeros( (_ACT_DIM,) )
    rtnVal = 0.0
    for i, frac in enumerate( fractV ):
        idx = indcsV[i]
        rtnAct += (A[ idx ] * frac)
        # print( V )
        rtnVal += (V[ idx ] * frac)
    return rtnAct, rtnVal
   

def eval_particle( state, action, value ):
    """ Decide whetner this point represents a particle worth saving """
    global N, F, V, A, KDT
    # 0. Get our estimate of the value of this state
    if N > 1:
        # print(N)
        estAct, estVal = get_action_and_value_inv_dist( state )
    else:
        estAct, estVal = None, None
    
    # 1. Find out if there is a particle there
    if KDT is not None:
        ndcs = KDT.query_ball_point( state, rad )
    else:
        ndcs = []

    # 2. If there is a particle already there and the current value is better, then update
    if len( ndcs ) and (value > estVal):
        index = ndcs[0]
        # fNear = points_from_indices( F, ndcs )
        vNear = points_from_indices( V, ndcs )
        if vNear[0] < value:
            A[index,:] = action
            V[index]   = value
        if len( ndcs ) > 1:
            print( "WARNING: NEARNESS CONSTRAINT VIOLATED" )

    # 3. Elif this is an open space that does NOT estimate the value well
    elif (estVal is None) or abs(estVal - value) > abs(value * vMar):
        # print( state, action, value )
        add_particle( state, action, value )
        recalc_spatial_tree()
    # Else this is an open space that predicts the value well, No update!
    # N. Return the current number of particles in the estimator
    return N
    
            

# Simple Learning Test

In [6]:
EPISODES = 1000
epLen    =  500
avg_time = 0
max_time = -1
env      = gym.make( 'CartPole-v1' ).env
env      = wrappers.RecordEpisodeStatistics( env, 100 )
div      = 10

In [7]:
for i_episode in range( EPISODES ):
    # instansiating the environment
    obs = env.reset()[0].reshape( (1,_OBS_DIM,) )
    print( f"Episode {i_episode+1}: Starting at {obs}" )
    for t in range( epLen ):
        # uncomment this is you want to see the rendering 
        #env.render()
        if (random() < eps) or (N<2):
            action = env.action_space.sample()
        else:
            action, preVal = get_action_and_value_inv_dist( obs )
            
        sLast = obs
        
        obs, reward, terminated, truncated, info = env.step( action )
        # print( obs, obs.shape )
        reward = cartpole_reward( obs )
        obs    = obs.reshape( (1,_OBS_DIM,) )
        eval_particle( sLast, action, reward )
        
        if terminated:
            avg_time = avg_time + t
            if t > max_time:
                max_time = t
                # print( f"\tMax. Uptime: {max_time}" )
            #print("Episode finished after {} timesteps".format(t+1))
            break
    # resetting the enviroment
    env.reset()
    if (i_episode>1) and ((i_episode%div) == 0):
        print( f'\navg time agent survives : {avg_time/i_episode}\n' )
        

# printing the avg time the game lasted
avg_time = avg_time/EPISODES
print( '\navg time agent survives :', avg_time, '\n' )

Episode 1: Starting at [[ 0.04585821 -0.03013133 -0.01170501  0.00973368]]
Episode 2: Starting at [[0.04277901 0.02966575 0.04554881 0.02560402]]
Episode 3: Starting at [[-0.00956217 -0.0200322   0.02313233 -0.01688593]]
Episode 4: Starting at [[ 0.04987307 -0.00032007  0.04040726  0.02645292]]
Episode 5: Starting at [[-0.02067019  0.00910813  0.01551832  0.04815488]]
Episode 6: Starting at [[-0.04852221 -0.01995321  0.04297921 -0.00264357]]
Episode 7: Starting at [[ 0.02015826  0.03430818 -0.03873846 -0.02855064]]
Episode 8: Starting at [[0.0419805  0.04407037 0.02538756 0.00818745]]
Episode 9: Starting at [[-0.03341462 -0.03477644 -0.03656272 -0.00576162]]
Episode 10: Starting at [[-0.01265672  0.00836684 -0.02485899  0.04109347]]
Episode 11: Starting at [[-0.00669007  0.00699021  0.00680441  0.00446752]]

avg time agent survives : 20.7

Episode 12: Starting at [[-0.03929454 -0.02699967  0.00410716  0.04727302]]
Episode 13: Starting at [[-0.00599468 -0.04957684  0.0124774   0.0494751